<a href="https://colab.research.google.com/github/butchland/fastai_xla_extensions/blob/master/explore_nbs/Demo_batch_tfms_TPU_performance_diff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### [RUNME] Install Colab compatible PyTorch/XLA wheels and dependencies
This may take up to ~2 minutes



In [ ]:
VERSION = "20200516"  #@param ["1.5" , "20200516", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  15835      0 --:--:-- --:--:-- --:--:-- 15835
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200516 ...
     |████████████████████████████████| 61kB 2.7MB/s 
Uninstalling torch-1.5.1+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.5.1+cu101
Uninstalling torchvision-0.6.1+cu101:
  Successfully uninstalled torchvision-0.6.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200516-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 91.0 MiB/ 91.0 MiB]                                                
Operation completed over 1 objects/91.0 MiB.                                     


In [ ]:
!pip install fastai2 --upgrade > /dev/null
!pip freeze | grep fastai2

fastai2==0.0.17


In [ ]:
!pip install git+https://github.com/butchland/fastai_xla_extensions > /dev/null
!pip freeze | grep fastai_xla_extensions

  Running command git clone -q https://github.com/butchland/fastai_xla_extensions /tmp/pip-req-build-dskrnjbj


In [ ]:
import fastai_xla_extensions.core

In [ ]:
from fastai2.vision.all import *

In [ ]:
path = untar_data(URLs.MNIST)

In [ ]:
Path.BASE_PATH = path; path.ls()

(#2) [Path('training'),Path('testing')]

In [ ]:
datablock = DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter=GrandparentSplitter(train_name='training',valid_name='testing'),
    item_tfms=Resize(28),
    batch_tfms=[]
)

In [ ]:
datablock_tfms= datablock.new(batch_tfms=[*aug_transforms(size=28, do_flip=False), Normalize],item_tfms=Resize(40))

In [ ]:
datablock_tfms.summary(path)

Setting-up type transforms pipelines
Found 70000 items
2 datasets of sizes 60000,10000
Setting up Pipeline: PILBase.create
Setting up Pipeline: parent_label -> Categorize

Building one sample
  Pipeline: PILBase.create
    starting from
      /root/.fastai/data/mnist_png/training/6/13035.png
    applying PILBase.create gives
      PILImage mode=RGB size=28x28
  Pipeline: parent_label -> Categorize
    starting from
      /root/.fastai/data/mnist_png/training/6/13035.png
    applying parent_label gives
      6
    applying Categorize gives
      TensorCategory(6)

Final sample: (PILImage mode=RGB size=28x28, TensorCategory(6))


Setting up after_item: Pipeline: Resize -> ToTensor
Setting up before_batch: Pipeline: 
Setting up after_batch: Pipeline: IntToFloatTensor -> AffineCoordTfm -> LightingTfm -> Normalize

Building one batch
Applying item_tfms to the first sample:
  Pipeline: Resize -> ToTensor
    starting from
      (PILImage mode=RGB size=28x28, TensorCategory(6))
    applying R

In [ ]:
dls1 = datablock_tfms.dataloaders(path,bs=256)
dls1.device

device(type='xla', index=1)

In [ ]:
learn1 = cnn_learner(dls1, resnet18, metrics=accuracy)

In [ ]:
learn1.fine_tune(0, freeze_epochs=4)

epoch,train_loss,valid_loss,accuracy,time
0,1.586431,0.894433,0.718600,02:00
1,0.999163,0.599764,0.804300,01:47
2,0.675172,0.407390,0.865100,01:46
3,0.455406,0.269324,0.913500,01:47


In [ ]:
datablock_no_tfms= datablock.new(batch_tfms=[],item_tfms=Resize(28))

In [ ]:
datablock_no_tfms.summary(path)

Setting-up type transforms pipelines
Found 70000 items
2 datasets of sizes 60000,10000
Setting up Pipeline: PILBase.create
Setting up Pipeline: parent_label -> Categorize

Building one sample
  Pipeline: PILBase.create
    starting from
      /root/.fastai/data/mnist_png/training/6/13035.png
    applying PILBase.create gives
      PILImage mode=RGB size=28x28
  Pipeline: parent_label -> Categorize
    starting from
      /root/.fastai/data/mnist_png/training/6/13035.png
    applying parent_label gives
      6
    applying Categorize gives
      TensorCategory(6)

Final sample: (PILImage mode=RGB size=28x28, TensorCategory(6))


Setting up after_item: Pipeline: Resize -> ToTensor
Setting up before_batch: Pipeline: 
Setting up after_batch: Pipeline: IntToFloatTensor

Building one batch
Applying item_tfms to the first sample:
  Pipeline: Resize -> ToTensor
    starting from
      (PILImage mode=RGB size=28x28, TensorCategory(6))
    applying Resize gives
      (PILImage mode=RGB size=28x2

In [ ]:
dls2 = datablock_no_tfms.dataloaders(path,bs=256)
dls2.device

device(type='xla', index=1)

In [ ]:
learn2 = cnn_learner(dls2, resnet18, metrics=accuracy)

In [ ]:
learn2.fine_tune(0, freeze_epochs=4)

epoch,train_loss,valid_loss,accuracy,time
0,1.417128,0.871235,0.730400,01:26
1,0.841705,0.563403,0.823100,01:23
2,0.549859,0.370359,0.880400,01:25
3,0.361350,0.258430,0.916000,01:25
